In [13]:
import networkx as nx
from sklearn.metrics import silhouette_score
from collections import defaultdict
import os
import numpy as np

# Funzione per caricare il grafo dal file GML
def load_graph_from_gml(gml_path):
    return nx.read_gml(gml_path)

# Funzione per recuperare le etichette di clustering dai nodi del grafo
def get_cluster_labels_from_graph(graph):
    labels = {}
    for node, data in graph.nodes(data=True):
        # Cerca prima 'cluster', poi 'community'
        cluster = data.get('cluster') or data.get('community')
        if cluster is not None:
            labels[node] = int(cluster)  # Converte l'etichetta in un intero
    return labels

# Funzione per calcolare la modularità del clustering
def compute_modularity(graph, cluster_labels):
    # Crea un dizionario per raggruppare i nodi per cluster
    clusters = defaultdict(set)
    for node, cluster in cluster_labels.items():
        clusters[cluster].add(node)
    
    # Crea una lista di insiemi di nodi per ciascun cluster
    community_list = list(clusters.values())
    try:
        return nx.community.modularity(graph, community_list)
    except Exception as e:
        print(f"Errore nel calcolo della modularità: {e}")
        return None

# Funzione per calcolare l'indice di silhouette
def compute_silhouette_score(graph, cluster_labels):
    # Ottieni le etichette dei cluster come una lista di interi
    clusters = list(cluster_labels.values())
    if len(set(clusters)) < 2:  # Silhouette non può essere calcolato con un solo cluster
        print("Silhouette Score non può essere calcolato con meno di 2 cluster.")
        return None
    
    # Estrai la matrice di adiacenza come matrice densa
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Calcola l'indice di silhouette
    try:
        return silhouette_score(adjacency_matrix, clusters, metric='precomputed')
    except ValueError as e:
        print(f"Errore nel calcolo della silhouette: {e}")
        return None

# Percorso della cartella che contiene i file GML generati dagli algoritmi
gml_folder_path = r'C:\Users\carcu\Desktop\Progetto Social\progetto_asnm-main\progetto_asnm-main\Code\notebook\Identificazione delle comunità'

# Recupera tutti i file GML dalla cartella
gml_files = [f for f in os.listdir(gml_folder_path) if f.endswith('.gml')]

# Confronta ogni algoritmo di clustering
for gml_file in gml_files:
    # Carica il grafo dal file GML
    graph = load_graph_from_gml(os.path.join(gml_folder_path, gml_file))
    
    # Recupera le etichette di clustering dal grafo
    cluster_labels = get_cluster_labels_from_graph(graph)
    
    if not cluster_labels:
        print(f"Attenzione: Nessuna etichetta trovata nel grafo {gml_file}.")
        print("="*50)
        continue
    
    # Calcola la modularità
    modularity_score = compute_modularity(graph, cluster_labels)
    
    # Calcola l'indice di silhouette
    silhouette_score_value = compute_silhouette_score(graph, cluster_labels)
    
    # Stampa i risultati
    print(f"Results for {gml_file}:")
    print(f"  Modularità: {modularity_score}")
    print(f"  Indice di Silhouette: {silhouette_score_value}")
    print("="*50)


Results for graph_with_affinity_propagation_clusters.gml:
  Modularità: 0.7963281120026618
  Indice di Silhouette: -0.6432244153259247
Results for graph_with_dbscan_clusters.gml:
  Modularità: 0.21815147495888076
  Indice di Silhouette: -0.5624481671919058
Results for graph_with_fcm_clusters.gml:
  Modularità: 0.023015172207017705
  Indice di Silhouette: -0.1924008621535241
Results for graph_with_girvan_newman_communities.gml:
  Modularità: 0.49997442760023203
  Indice di Silhouette: -0.731796317797313
Results for graph_with_gmm_clusters.gml:
  Modularità: 0.00035965967503180326
  Indice di Silhouette: -0.6399071156078951
Results for graph_with_kmeans_clusters.gml:
  Modularità: -3.2717109059920497e-05
  Indice di Silhouette: -0.652015259578703
Results for graph_with_label_propagation_clusters.gml:
  Modularità: 0.8238501326356934
  Indice di Silhouette: -0.6107148780892354
Results for graph_with_leiden_clusters.gml:
  Modularità: 0.9180793678216606
  Indice di Silhouette: -0.648863824

In [24]:
import sys
import networkx as nx
from networkx.algorithms.community import modularity
from sklearn.metrics import silhouette_score
from collections import defaultdict
import os

# Funzione per caricare il grafo dal file GML
def load_graph_from_gml(gml_path):
    return nx.read_gml(gml_path)

# Funzione per recuperare le etichette di clustering dai nodi del grafo
def get_cluster_labels_from_graph(graph):
    labels = {}
    for node, data in graph.nodes(data=True):
        # Cerca prima 'cluster', poi 'community'
        cluster = data.get('cluster') or data.get('community')
        if cluster is not None:
            labels[node] = int(cluster)  # Converte l'etichetta in un intero
    return labels

# Funzione per calcolare la modularità del clustering
def compute_modularity(graph, cluster_labels):
    # Crea un dizionario per raggruppare i nodi per cluster
    clusters = defaultdict(set)
    for node, cluster in cluster_labels.items():
        clusters[cluster].add(node)
    
    # Crea una lista di insiemi di nodi per ciascun cluster
    community_list = list(clusters.values())
    
    # Verifica che ci siano almeno 2 comunità
    if len(community_list) < 2:
        print(f"Errore: il grafo ha meno di 2 comunità. Modularità non calcolabile.")
        return 0
    
    try:
        return nx.community.modularity(graph, community_list)
    except Exception as e:
        print(f"Errore nel calcolo della modularità: {e}")
        return None

# Funzione per calcolare l'indice di silhouette
def compute_silhouette_score(graph, cluster_labels):
    # Ottieni le etichette dei cluster come una lista di interi
    clusters = list(cluster_labels.values())
    if len(set(clusters)) < 2:  # Silhouette non può essere calcolato con un solo cluster
        print("Silhouette Score non può essere calcolato con meno di 2 cluster.")
        return None
    
    # Estrai la matrice di adiacenza come matrice densa
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Calcola l'indice di silhouette
    try:
        return silhouette_score(adjacency_matrix, clusters, metric='precomputed')
    except ValueError as e:
        print(f"Errore nel calcolo della silhouette: {e}")
        return None

# Funzione per eseguire Girvan-Newman (Newman-Girvan) sul grafo
def girvan_newman(graph, max_communities=5, centrality_threshold=None):
    graph_copy = graph.copy()
    while len(list(nx.connected_components(graph_copy))) < max_communities:
        edge_betweenness = nx.edge_betweenness_centrality(graph_copy)
        
        if centrality_threshold:
            edges_to_remove = [
                edge for edge, centrality in edge_betweenness.items() if centrality > centrality_threshold
            ]
        else:
            edge_to_remove = max(edge_betweenness, key=edge_betweenness.get)
            edges_to_remove = [edge_to_remove]
        
        if not edges_to_remove:
            break
        
        graph_copy.remove_edges_from(edges_to_remove)
    
    communities = [set(component) for component in nx.connected_components(graph_copy)]
    return communities

# Percorso della cartella che contiene i file GML generati dagli algoritmi
gml_folder_path = r'C:\Users\carcu\Desktop\Progetto Social\progetto_asnm-main\progetto_asnm-main\Code\notebook\Identificazione delle comunità'

# Recupera tutti i file GML dalla cartella
gml_files = [f for f in os.listdir(gml_folder_path) if f.endswith('.gml')]

# Confronta ogni algoritmo di clustering
for gml_file in gml_files:
    # Carica il grafo dal file GML
    graph = load_graph_from_gml(os.path.join(gml_folder_path, gml_file))
    
    # Recupera le etichette di clustering dal grafo
    cluster_labels = get_cluster_labels_from_graph(graph)
    
    if not cluster_labels:
        print(f"Attenzione: Nessuna etichetta trovata nel grafo {gml_file}.")
        print("="*50)
        continue
    
    # Verifica che le etichette siano valide
    print(f"Verifica etichette per {gml_file}: {cluster_labels}")
    
    # Calcola la modularità
    modularity_score = compute_modularity(graph, cluster_labels)
    
    # Calcola l'indice di silhouette
    silhouette_score_value = compute_silhouette_score(graph, cluster_labels)
    
    # Stampa solo i risultati
    print(f"Results for {gml_file}:")
    print(f"  Modularità: {modularity_score}")
    print(f"  Indice di Silhouette: {silhouette_score_value}")
    print("="*50)


Verifica etichette per graph_with_affinity_propagation_clusters.gml: {'62926993725': 0, '531758406': 0, '60850232106': 0, '55968501995': 0, '60628346710': 0, '65391034968': 0, '38077850387': 1, '55178050501': 1, '54579428206': 1, '64379260940': 1, '262856836': 1, '56714285656': 1, '34158564957': 1, '50656527917': 1, '62233317658': 1, '1501070805': 2, '1795971330': 2, '3098195771': 2, '33498315874': 2, '65532133383': 3, '51524079790': 3, '65242802457': 3, '58086058382': 4, '67044607463': 4, '69892071057': 329, '53982955686': 3, '62619594223': 3, '69474045690': 160, '6267328541': 3, '65465879934': 197, '67115637172': 3, '69807941929': 183, '70119456026': 72, '68982694141': 169, '65236883921': 40, '67038712330': 3, '67281319268': 3, '65360210258': 3, '67592464790': 3, '59097305524': 40, '65448076319': 331, '69511573422': 49, '69002489097': 3, '66867106474': 329, '24736383042': 5, '70690130037': 5, '49519786147': 5, '4023341285': 5, '6053618475': 5, '69822893255': 5, '63959794645': 5, '861

In [21]:
import networkx as nx
from sklearn.metrics import silhouette_score
import os
import numpy as np

# Funzione per caricare il grafo dal file GML
def load_graph_from_gml(gml_path):
    return nx.read_gml(gml_path)

# Funzione per recuperare le etichette di clustering dai nodi del grafo
def get_cluster_labels_from_graph(graph):
    labels = {}
    for node, data in graph.nodes(data=True):
        # Supponiamo che l'attributo 'cluster' contenga l'etichetta del cluster
        labels[node] = data.get('cluster', None)  # Assicurati che il nome dell'attributo sia corretto
    return labels

# Funzione per calcolare la modularità del clustering
def compute_modularity(graph, cluster_labels):
    # Crea un dizionario per raggruppare i nodi per cluster
    clusters = {}
    for node, cluster in cluster_labels.items():
        if cluster not in clusters:
            clusters[cluster] = set()
        clusters[cluster].add(node)
    
    # Crea una lista di insiemi di nodi per ciascun cluster
    community_list = list(clusters.values())
    return nx.community.modularity(graph, community_list)

# Funzione per calcolare l'indice di silhouette
def compute_silhouette_score(graph, cluster_labels):
    # Ottieni le etichette dei cluster come una lista di interi
    clusters = list(cluster_labels.values())
    
    # Estrai le relazioni di connessione tra i nodi
    adjacency_matrix = nx.to_numpy_array(graph)  # Cambiato da to_numpy_matrix a to_numpy_array
    
    # Calcola l'indice di silhouette
    try:
        return silhouette_score(adjacency_matrix, clusters, metric='precomputed')
    except ValueError:
        return None  # Ritorna None se non è possibile calcolare l'indice di silhouette

# Percorso della cartella che contiene i file GML generati dagli algoritmi
gml_folder_path = r'C:\Users\carcu\Desktop\Progetto Social\progetto_asnm-main\progetto_asnm-main\Code\notebook\Identificazione delle comunità'

# Recupera tutti i file GML dalla cartella
gml_files = [f for f in os.listdir(gml_folder_path) if f.endswith('.gml')]

# Confronta ogni algoritmo di clustering
for gml_file in gml_files:
    # Carica il grafo dal file GML
    graph = load_graph_from_gml(os.path.join(gml_folder_path, gml_file))
    
    # Recupera le etichette di clustering dal grafo
    cluster_labels = get_cluster_labels_from_graph(graph)
    
    # Verifica che ci siano etichette nel grafo
    if not cluster_labels:
        print(f"Attenzione: Nessuna etichetta trovata nel grafo {gml_file}.")
        print("="*50)
        continue
    
    # Calcola la modularità
    modularity_score = compute_modularity(graph, cluster_labels)
    
    # Calcola l'indice di silhouette
    silhouette_score_value = compute_silhouette_score(graph, cluster_labels)
    
    # Stampa i risultati
    print(f"Results for {gml_file}:")
    print(f"  Modularità: {modularity_score}")
    print(f"  Indice di Silhouette: {silhouette_score_value}")
    print("="*50)


Results for graph_with_affinity_propagation_clusters.gml:
  Modularità: 0.7963281120026618
  Indice di Silhouette: -0.6432244153259247
Results for graph_with_dbscan_clusters.gml:
  Modularità: 0.21815147495888076
  Indice di Silhouette: -0.5624481671919058
Results for graph_with_fcm_clusters.gml:
  Modularità: 0.023015172207017705
  Indice di Silhouette: -0.1924008621535241
Results for graph_with_girvan_newman_communities.gml:
  Modularità: 0.0
  Indice di Silhouette: None
Results for graph_with_gmm_clusters.gml:
  Modularità: 0.00035965967503180326
  Indice di Silhouette: -0.6399071156078951
Results for graph_with_kmeans_clusters.gml:
  Modularità: -3.2717109059920497e-05
  Indice di Silhouette: -0.652015259578703
Results for graph_with_label_propagation_clusters.gml:
  Modularità: 0.8238501326356934
  Indice di Silhouette: -0.6107148780892354
Results for graph_with_leiden_clusters.gml:
  Modularità: 0.9180793678216606
  Indice di Silhouette: -0.6488638248465749
Results for graph_with